In [1]:
import os
import sys
from pathlib import Path

def setup_project_imports(include_direct_imports=False):
    """
    Sets up the Python path to allow imports from project root.
    Args:
        include_direct_imports: If True, allows importing without the scripts/agents prefix
    """
    # Get the notebook's directory and project root
    current_dir = Path(os.getcwd())
    project_root = str(current_dir.parent)
    scripts_dir = os.path.join(project_root, 'scripts')
    
    # Clean up path
    for path in [str(current_dir), project_root, scripts_dir]:
        if path in sys.path:
            sys.path.remove(path)
    
    # Add paths
    sys.path.insert(0, project_root)
    if include_direct_imports:
        sys.path.insert(0, scripts_dir)  # This allows direct imports
    
    print("✅ Import paths configured:")
    if include_direct_imports:
        print("   You can use: from items import Item")
    print("   You can use: from scripts.items import Item")

# Run setup with direct imports enabled
setup_project_imports(include_direct_imports=True)

✅ Import paths configured:
   You can use: from items import Item
   You can use: from scripts.items import Item


In [5]:
# imports

import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from agents.deals import ScrapedDeal, DealSelection

In [6]:
# Initialize and constants

load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
MODEL = 'gpt-4o-mini'
openai = OpenAI()

In [7]:
deals = ScrapedDeal.fetch(show_progress=True)

100%|██████████| 5/5 [07:05<00:00, 85.05s/it]


In [8]:
len(deals)

50

In [9]:
deals[5].describe()

'Title: TCL TV Deals at Best Buy: Up to 1,700 off + free shipping\nDetails: Save on nearly 30 models all 65" or larger. We\'ve pictured the best selling TCL QM7-Series 65" 4K HDR QD-Mini LED UHD Smart TV for $700 ($400 off). Shop Now at Best Buy\nFeatures: \nURL: https://www.dealnews.com/TCL-TV-Deals-at-Best-Buy-Up-to-1-700-off-free-shipping/21713311.html?iref=rss-c142'

In [10]:
system_prompt = """You identify and summarize the 5 most detailed deals from a list, by selecting deals that have the most detailed, high quality description and the most clear price.
Respond strictly in JSON with no explanation, using this format. You should provide the price as a number derived from the description. If the price of a deal isn't clear, do not include that deal in your response.
Most important is that you respond with the 5 deals that have the most detailed product description with price. It's not important to mention the terms of the deal; most important is a thorough description of the product.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

{"deals": [
    {
        "product_description": "Your clearly expressed summary of the product in 4-5 sentences. Details of the item are much more important than why it's a good deal. Avoid mentioning discounts and coupons; focus on the item itself. There should be a paragpraph of text for each item you choose.",
        "price": 99.99,
        "url": "the url as provided"
    },
    ...
]}"""

In [11]:
user_prompt = """Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

"""
user_prompt += '\n\n'.join([deal.describe() for deal in deals])

In [12]:
print(user_prompt[:2000])

Respond with the most promising 5 deals from this list, selecting those which have the most detailed, high quality product description and a clear price.
Respond strictly in JSON, and only JSON. You should rephrase the description to be a summary of the product itself, not the terms of the deal.
Remember to respond with a paragraph of text in the product_description field for each of the 5 items that you select.
Be careful with products that are described as "$XXX off" or "reduced by $XXX" - this isn't the actual price of the product. Only respond with products when you are highly confident about the price. 

Deals:

Title: Logitech G560 PC Gaming Speaker System for $180 + free shipping
Details: It's $20 off and the lowest price we could find. Buy Now at Best Buy
Features: 7.1 DTS:X Ultra Surround Sound  Game based LIGHTSYNC RGB Two Speakers and Subwoofer Bluetooth and USB Model: 980-001300
URL: https://www.dealnews.com/products/Logitech/Logitech-G560-PC-Gaming-Speaker-System/486469.ht

In [13]:
def get_recommendations():
    completion = openai.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt}
      ],
        response_format=DealSelection
    )
    result = completion.choices[0].message.parsed
    return result

In [14]:
result = get_recommendations()

In [17]:
type(result)

agents.deals.DealSelection

In [15]:
len(result.deals)

5

In [16]:
result.deals[1]

Deal(product_description='The Unlocked Google Pixel 9 Pro Fold Android Smartphone boasts a stunning 6.3" 1080x2424 OLED display and features an internal 8" Super Actua Flex display. It is equipped with a 10MP dual selfie camera and a robust rear camera setup of 48MP wide, 10MP dual ultrawide, and 10.8MP dual telephoto options. With 16GB of RAM, this phone runs on Android 14 and includes innovative Gemini AI photo tools.', price=1399.0, url='https://www.dealnews.com/products/Google/Unlocked-Google-Pixel-9-Pro-Fold-Android-Smartphone/486447.html?iref=rss-c142')

In [19]:
from agents.scanner_agent import ScannerAgent

In [20]:
agent = ScannerAgent()
result = agent.scan()

In [22]:
print(result)

deals=[Deal(product_description="The Logitech G560 PC Gaming Speaker System delivers immersive 7.1 DTS:X Ultra Surround Sound, making it a perfect companion for gamers. This speaker system integrates Game-based LIGHTSYNC RGB lighting that responds to the action on your screen, enhancing your gaming experience. Designed with a two-speaker and subwoofer setup, it's compatible with Bluetooth and USB connections, giving you flexibility and ease of use. The model number for this high-performance audio solution is 980-001300.", price=180.0, url='https://www.dealnews.com/products/Logitech/Logitech-G560-PC-Gaming-Speaker-System/486469.html?iref=rss-c142'), Deal(product_description='The Unlocked Google Pixel 9 Pro Fold Android Smartphone impresses with its stunning 6.3-inch 1080x2424 OLED display and remarkable 8-inch internal Super Actua Flex display. Photography enthusiasts will appreciate the sophisticated camera system, featuring a 10MP dual selfie camera and a versatile rear setup with a 4